# 1. Dense Encoder로 문장 벡터화

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# 사용할 모델과 토크나이저를 로드
model_name = "bert-base-multilingual-cased"  # 다국어 BERT 모델
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 문장 벡터 생성 함수
def encode_sentences(sentences):
    # 토큰화 및 텐서로 변환
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():  # 그래디언트 계산 비활성화
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].numpy()  # [CLS] 토큰의 출력을 사용

# text 컬럼으로부터 문장 벡터화
sentence_vectors = encode_sentences(cleaned_df['text'].tolist())


# 2. t-SNE를 이용한 시각화

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# t-SNE를 사용하여 차원 축소
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
tsne_results = tsne.fit_transform(sentence_vectors)

# t-SNE 결과 시각화
plt.figure(figsize=(12, 8))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=cleaned_df['cluster'], cmap='viridis', alpha=0.5)
plt.colorbar(label='Cluster')
plt.title('t-SNE Visualization of Sentence Clusters')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.grid()
plt.show()


# 3. UMAP을 이용한 시각화 (선택 사항)

In [ ]:
import umap

# UMAP을 사용하여 차원 축소
umap_model = umap.UMAP(n_neighbors=15, n_components=2, metric='euclidean')
umap_results = umap_model.fit_transform(sentence_vectors)

# UMAP 결과 시각화
plt.figure(figsize=(12, 8))
plt.scatter(umap_results[:, 0], umap_results[:, 1], c=cleaned_df['cluster'], cmap='viridis', alpha=0.5)
plt.colorbar(label='Cluster')
plt.title('UMAP Visualization of Sentence Clusters')
plt.xlabel('UMAP Component 1')
plt.ylabel('UMAP Component 2')
plt.grid()
plt.show()
